In [1]:
# Lab 11 MNIST and Deep learning CNN
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# gpu 쓸 수 있으면 gpu 쓰고 안되면 cpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset 가져오기
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader. DataLoader를 이용해 학습을 효율적으로 진행할 수 있다
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), # kernel 개수, 사이즈(정사각형이라 n*n 사이즈 커널을 사용), 연산 후 이동 간격, padding 정도(외곽에 padding을 얼마나 해줄건가)
            torch.nn.ReLU(), # 활성화 함수
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # kerner_size * kernel_size 영역별로 가장 큰 값만 다음 레이어로 넘김
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # 입력받는 데이터의 채널 개수, kernel 개수, kernel 크기, 연산 후 이동 간격, padding 정도
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs. 다차원 입력값을 일렬로 늘려서(Flatten) 선형 연산 수행
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs. 선형 연산 끝에 10개의 출력값 휙득
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    # 정전파 
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [7]:
# instantiate CNN model
model = CNN().to(device)

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed. device에서 로스함수를 돌리겠다는 뜻
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 최적화 방법을 Adam으로

In [9]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        # 디바이스에 X, Y 데이터 할당
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad() # 그레디언트 초기화 
        hypothesis = model(X) # 정전파
        cost = criterion(hypothesis, Y) # 로스 구하기 
        cost.backward() # 역전파로 그레디언트 구하기
        optimizer.step() # 구한 그레디언트로 optimizer -> 가중치 업데이ㅡㅌ

        avg_cost += cost / total_batch # 평균 로스 구하기 위함

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.192344591
[Epoch:    2] cost = 0.0547086298
[Epoch:    3] cost = 0.0385811999
[Epoch:    4] cost = 0.0303558167
[Epoch:    5] cost = 0.0259795897
[Epoch:    6] cost = 0.020061627
[Epoch:    7] cost = 0.0168916639
[Epoch:    8] cost = 0.0149363596
[Epoch:    9] cost = 0.0155974198
[Epoch:   10] cost = 0.0110683329
[Epoch:   11] cost = 0.0108940834
[Epoch:   12] cost = 0.0106782485
[Epoch:   13] cost = 0.00867550541
[Epoch:   14] cost = 0.00709024211
[Epoch:   15] cost = 0.00782931503
Learning Finished!


In [10]:
# Test model and check accuracy
# 모델 테스트, 정확도 휙득
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False). 이렇게 해야 학습할 때만 사용하는 dropout, Batch Normalization 등을 사용하지 않는다

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9902999997138977
